# This is the imports section

In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import time,csv,sys,pytz,os
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select,WebDriverWait

import numpy as np
from datetime import date,datetime,timedelta

In [3]:
# refer following discussion
# https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path
# install chromium, its driver, and selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

# !pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# # open it, go to a website, and get results
# wd = webdriver.Chrome('chromedriver',options=options)
# wd.get("https://www.website.com")
# print(wd.page_source)  # results

In [4]:
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
%load_ext dotenv
%dotenv

cannot find .env file


In [6]:
!pip install py2neo -q

# Connect to db instance then init the graph object with py2neo

In [7]:
from py2neo import Graph, Node, Relationship

g = Graph(f"neo4j+s://8686653a.databases.neo4j.io", auth=("neo4j", 'gzZr3zneKvXdUPc3_hO_uj_KQOy3g3zTGNal3TmCvhs'), routing=True)

In [8]:
print(g.run("MATCH (n) RETURN n"))

 n                                                                          
----------------------------------------------------------------------------
 (_0:Tag {title: 'Array', uri: 'https://leetcode.com/tag/array'})           
 (_1:Tag {title: 'String', uri: 'https://leetcode.com/tag/string'})         
 (_2:Tag {title: 'Hash Table', uri: 'https://leetcode.com/tag/hash-table'}) 



In [9]:
g.nodes.match("Problem",uri='https://leetcode.com/problems/two-sum/').first()

Node('Problem', acceptance='49.1%', difficulty='Easy', finished='Done', title='1. Two Sum', uri='https://leetcode.com/problems/two-sum/')

# Initialize the leetcode API

In [10]:
!pip install python-leetcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import leetcode

leetcode_session = ""
configuration = leetcode.Configuration()
csrf_token = ""
configuration.api_key["x-csrftoken"] = 'IZXaZ2dS0FIKkPuH6BqTB1Mbd3UBigk1ptI77U3J3PYPFia8B7Ee6Ewxeeb2HefZ'
configuration.api_key["csrftoken"] = 'IZXaZ2dS0FIKkPuH6BqTB1Mbd3UBigk1ptI77U3J3PYPFia8B7Ee6Ewxeeb2HefZ'
configuration.api_key["LEETCODE_SESSION"] = 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJfYXV0aF91c2VyX2lkIjoiNDg4MTc1OSIsIl9hdXRoX3VzZXJfYmFja2VuZCI6ImFsbGF1dGguYWNjb3VudC5hdXRoX2JhY2tlbmRzLkF1dGhlbnRpY2F0aW9uQmFja2VuZCIsIl9hdXRoX3VzZXJfaGFzaCI6IjI3ZjczNGE1ZWJjOTNiYjcxMGM2YzU1YTY1OWUzOWFhMzg0ZTdjN2YiLCJpZCI6NDg4MTc1OSwiZW1haWwiOiJkZXZ5YW5iaUB1c2MuZWR1IiwidXNlcm5hbWUiOiJkYmlzd2FzIiwidXNlcl9zbHVnIjoiZGJpc3dhcyIsImF2YXRhciI6Imh0dHBzOi8vczMtdXMtd2VzdC0xLmFtYXpvbmF3cy5jb20vczMtbGMtdXBsb2FkL2Fzc2V0cy9kZWZhdWx0X2F2YXRhci5qcGciLCJyZWZyZXNoZWRfYXQiOjE2NjkyNTU1NjMsImlwIjoiMjYwMTo2NDc6NDYwMTo1MmQwOmQwNWE6ZTA3YjpjNGRkOmQ0MzYiLCJpZGVudGl0eSI6ImYzOTk3YzIyNjgzOTNjM2U0NGRiOWQ2MTcxYzdkMTkyIiwic2Vzc2lvbl9pZCI6MzEyMjA5MzAsIl9zZXNzaW9uX2V4cGlyeSI6MTIwOTYwMH0.0PK68imJvkrsd-pXCzKLWVAQ_x0lIQnp9g3kDj3XOYM'
configuration.api_key["Referer"] = "https://leetcode.com"
configuration.debug = False

In [12]:
api_instance =leetcode.DefaultApi(leetcode.ApiClient(configuration))

## Quick test of API

In [13]:
graphql_request = leetcode.GraphqlQuery(
query="""
      {
        user {
            username
            isCurrentUserPremium
        }
      }
""",
variables=leetcode.GraphqlQueryVariables(),
)
print(api_instance.graphql_post(body=graphql_request))

{'data': {'question': None,
          'user': {'is_current_user_premium': True, 'username': 'dbiswas'}}}


# Get a list of the user's completed and attempted problems
- NOTE: This is a subset, as the python leetcode api is incomplete and i couldn't figure out how to get it from all topics lell

In [14]:
api_response=api_instance.api_problems_topic_get(topic="algorithms")
solved_questions=list()
attempted_questions=list()
# print(len(api_response.stat_status_pairs))
for questions in api_response.stat_status_pairs:
  # print(questions)
  item_entry = dict()
  if questions.status=="ac":
    item_entry['title'] = questions.stat.question__title
    item_entry['id'] = questions.stat.question_id
    item_entry['difficulty'] = questions.difficulty
    item_entry['frequency'] = questions.frequency
    solved_questions.append(item_entry)
  if questions.status=="notac":
    item_entry['title'] = questions.stat.question__title
    item_entry['id'] = questions.stat.question_id
    item_entry['difficulty'] = questions.difficulty
    item_entry['frequency'] = questions.frequency
    attempted_questions.append(item_entry)


print(solved_questions)
print(attempted_questions)
print("Total number of solved questions ",len(solved_questions))
print("Total number of attempted questions ",len(attempted_questions))

[{'title': 'Group Anagrams', 'id': 49, 'difficulty': {'level': 2}, 'frequency': 7.19771305045021}, {'title': 'Valid Parentheses', 'id': 20, 'difficulty': {'level': 1}, 'frequency': 7.7613083066231034}, {'title': 'Longest Common Prefix', 'id': 14, 'difficulty': {'level': 1}, 'frequency': 7.405713224084088}, {'title': 'Add Two Numbers', 'id': 2, 'difficulty': {'level': 2}, 'frequency': 7.643929542203227}, {'title': 'Two Sum', 'id': 1, 'difficulty': {'level': 1}, 'frequency': 8.083495977875183}]
[{'title': 'Palindrome Number', 'id': 9, 'difficulty': {'level': 1}, 'frequency': 7.058118118234015}]
Total number of solved questions  5
Total number of attempted questions  1


# First Recommendation style:
- Similar problems + difficulty

In [15]:
diff_map = {
    1: 'Easy',
    2: 'Medium',
    3: 'Hard'
}

In [16]:
# Get the diffficulty of problems done
diffs_list = {}
diffs_list['Easy'] = 0
diffs_list['Medium'] = 0
diffs_list['Hard'] = 0
for problem in solved_questions:
  diffs_list[diff_map[problem['difficulty'].level]] += 1
print(diffs_list)

{'Easy': 3, 'Medium': 2, 'Hard': 0}


In [17]:
diffs_list = {k: v for k, v in sorted(diffs_list.items(), key=lambda item: item[1])}
diffs_list

{'Hard': 0, 'Medium': 2, 'Easy': 3}

In [18]:
problem_mapping = {}
hard_coded_problems = [3,4,3]
i=0
for item in diffs_list:
  problem_mapping[item] = hard_coded_problems[i]
  i = i + 1
problem_mapping

{'Hard': 3, 'Medium': 4, 'Easy': 3}

In [19]:
hard = problem_mapping['Hard']
medium = problem_mapping['Medium']
easy = problem_mapping['Easy']
print(hard, medium, easy)

3 4 3


In [20]:
# For each question in solved, look at the linked problems and add to a list
candidate_problems = []
for problem in solved_questions:
  prob_string = str(problem['id']) + ". " + problem['title']
  # print(prob_string)
  results = g.run(f"MATCH (n:Problem)-->(t:Problem) WHERE n.title = '{prob_string}' RETURN *")
  results = results.data()
  if(len(results)) == 0:
    continue
  for item in results:
    candidate_problems.append(item['t'])

In [21]:
# Split the similar problems into easy, medium, hard
easy_problems = list()
medium_problems = list()
hard_problems = list()
for c_prob in candidate_problems:
  diff = c_prob['difficulty']
  if diff == 'Easy':
    easy_problems.append(c_prob)
  if diff == 'Medium':
    medium_problems.append(c_prob)
  if diff == 'Hard':
    hard_problems.append(c_prob)

In [22]:
# Select the number of easy, medium, and hard problems defined dabv at random
import random
random.seed(11)
ez_probs = random.choices(easy_problems, k=easy)
med_probs = random.choices(medium_problems, k=medium)
hrd_probs = random.choices(hard_problems, k=hard)

In [23]:
urls = list()
for i in ez_probs:
  urls.append(i['uri'])
for i in med_probs:
  urls.append(i['uri'])
for i in hrd_probs:
  urls.append(i['uri'])
print(urls)

['https://leetcode.com/problems/find-subarrays-with-equal-sum/', 'https://leetcode.com/problems/check-distances-between-same-letters/', 'https://leetcode.com/problems/two-sum-iv-input-is-a-bst/', 'https://leetcode.com/problems/multiply-strings/', 'https://leetcode.com/problems/node-with-highest-edge-score/', 'https://leetcode.com/problems/number-of-pairs-of-strings-with-concatenation-equal-to-target/', 'https://leetcode.com/problems/check-if-a-parentheses-string-can-be-valid/', 'https://leetcode.com/problems/longest-valid-parentheses/', 'https://leetcode.com/problems/longest-valid-parentheses/', 'https://leetcode.com/problems/number-of-excellent-pairs/']


# Recommendation method 2:
- Similar questions + tags

In [24]:
# For each question in solved, look at the linked tags and add them to a count dict
tag_counts = {}
for problem in solved_questions:
  prob_string = str(problem['id']) + ". " + problem['title']
  # print(prob_string)
  result_tags = g.run(f"MATCH (n:Problem)-->(p:Tag) WHERE n.title = '{prob_string}' RETURN p")
  # results = g.run(f"MATCH (n:Problem)-->(t:Problem) WHERE n.title = '{prob_string}' RETURN *")
  tags = result_tags.data()
  if(len(tags)) == 0:
    continue
  # print(tags)
  for result in tags:
    tag = result['p']['title']
    if tag not in tag_counts:
      tag_counts[tag] = 1
    else:
      tag_counts[tag] += 1

print(tag_counts)

{'Sorting': 1, 'String': 3, 'Hash Table': 2, 'Array': 2, 'Stack': 1, 'Recursion': 1, 'Math': 1, 'Linked List': 1}


In [25]:
# We look at the tag nodes of linked problems and, depending on the probabilities set,
# we add em
tag_to_problems = {}
for problem in solved_questions:
  prob_string = str(problem['id']) + ". " + problem['title']
  linked_problems_tags = g.run(f"MATCH (n:Problem)-->(p:Tag)<--(t:Problem) WHERE n.title = '{prob_string}' RETURN p,t")
  for pair in linked_problems_tags.data():
    if pair['p']['title'] not in tag_to_problems:
      tag_to_problems[pair['p']['title']] = list()
      tag_to_problems[pair['p']['title']].append(pair['t'])
    else:
      tag_to_problems[pair['p']['title']].append(pair['t'])

In [26]:
for item in tag_to_problems:
  print(item)

Sorting
String
Hash Table
Array
Stack
Recursion
Math
Linked List


In [27]:
tag_counts = {k: v for k, v in sorted(tag_counts.items(), key=lambda item: item[1])}
tag_counts

{'Sorting': 1,
 'Stack': 1,
 'Recursion': 1,
 'Math': 1,
 'Linked List': 1,
 'Hash Table': 2,
 'Array': 2,
 'String': 3}

In [28]:
random.seed(12)
total = 0
max_rec = 10
recs = list()
for item in tag_counts:
  if len(recs) >= max_rec:
    break
  opts = tag_to_problems[item]
  problem = random.choices(opts, k=1)[0]
  recs.append(problem['uri'])

random.seed(11)
if len(recs) < max_rec:
  for item in tag_counts:
    if len(recs) >= max_rec:
      break
    opts = tag_to_problems[item]
    problem = random.choices(opts, k=1)[0]
    recs.append(problem['uri'])

print(recs)

['https://leetcode.com/problems/sell-diminishing-valued-colored-balls/', 'https://leetcode.com/problems/exclusive-time-of-functions/', 'https://leetcode.com/problems/reorder-list/', 'https://leetcode.com/problems/strictly-palindromic-number/', 'https://leetcode.com/problems/steps-to-make-array-non-decreasing/', 'https://leetcode.com/problems/k-diff-pairs-in-an-array/', 'https://leetcode.com/problems/rank-transform-of-an-array/', 'https://leetcode.com/problems/iterator-for-combination/', 'https://leetcode.com/problems/maximum-height-by-stacking-cuboids/', 'https://leetcode.com/problems/backspace-string-compare/']


#KG Eval Section

In [29]:
total_nodes = g.run("MATCH (n) RETURN n")

In [30]:
total_nodes = total_nodes.data()

In [31]:
print(len(total_nodes))

2549


## B/t Solved problems

In [32]:
# Solved-Solved Evaluation Metric
for i in range(0, len(solved_questions)-1):
    # inner loop
    for j in range(i+1, len(solved_questions)):
      source = solved_questions[i]
      source = str(source['id']) + ". " + source['title']
      target = solved_questions[j]
      target = str(target['id']) + ". " + target['title']

      func_string_a = f""" 
        MATCH p=shortestPath((t:Problem {{title: '{source}'}})-[:SIMILAR_TO|BELONGS_TO*1..100]-(n:Problem {{title: '{target}'}}))
        RETURN distinct p
      """

      path_data = g.run(func_string_a)
      path_dist = path_data.data()
      for pair in path_dist:
        path_nodes_string = ""  
        for nodes in pair['p'].nodes:
          path_nodes_string = path_nodes_string + ", " + nodes['title']
        print("Shortest path:", path_nodes_string)
        print("Path Length = ", len(pair['p'].nodes)-1)
      print("===================")

Shortest path: , 49. Group Anagrams, String, 20. Valid Parentheses
Path Length =  2
Shortest path: , 49. Group Anagrams, String, 14. Longest Common Prefix
Path Length =  2
Shortest path: , 49. Group Anagrams, Array, 989. Add to Array-Form of Integer, 2. Add Two Numbers
Path Length =  3
Shortest path: , 49. Group Anagrams, Hash Table, 1. Two Sum
Path Length =  2
Shortest path: , 20. Valid Parentheses, String, 14. Longest Common Prefix
Path Length =  2
Shortest path: , 20. Valid Parentheses, 22. Generate Parentheses, Recursion, 2. Add Two Numbers
Path Length =  3
Shortest path: , 20. Valid Parentheses, 2337. Move Pieces to Obtain a String, Two Pointers, 18. 4Sum, 1. Two Sum
Path Length =  4
Shortest path: , 14. Longest Common Prefix, String, 43. Multiply Strings, 2. Add Two Numbers
Path Length =  3
Shortest path: , 14. Longest Common Prefix, String, 288. Unique Word Abbreviation, 170. Two Sum III - Data structure design, 1. Two Sum
Path Length =  4
Shortest path: , 2. Add Two Numbers, 98

## B/t a control random and solved problems

In [33]:
# Random-Solved Evaluation Metric
# Node Chose: Problem 2399. Check Distances Between Same Letters 
control_problem = '2399. Check Distances Between Same Letters'
for problem in solved_questions:
      target = str(problem['id']) + ". " + problem['title']
      func_string_b = f""" 
        MATCH p=shortestPath((t:Problem {{title: '{control_problem}'}})-[:SIMILAR_TO|BELONGS_TO*1..100]-(n:Problem {{title: '{target}'}}))
        RETURN distinct p
      """
      path_data = g.run(func_string_b)
      path_dist = path_data.data()

      for pair in path_dist:
        path_nodes_string = ""  
        for nodes in pair['p'].nodes:
          path_nodes_string = path_nodes_string + ", " + nodes['title']
        print("Shortest path:", path_nodes_string)
        print("Path Length = ", len(pair['p'].nodes)-1)
      print("===================")

Shortest path: , 2399. Check Distances Between Same Letters, 1. Two Sum, Hash Table, 49. Group Anagrams
Path Length =  3
Shortest path: , 2399. Check Distances Between Same Letters, 1. Two Sum, 18. 4Sum, Two Pointers, 2337. Move Pieces to Obtain a String, 20. Valid Parentheses
Path Length =  5
Shortest path: , 2399. Check Distances Between Same Letters, 1. Two Sum, Hash Table, 648. Replace Words, String, 14. Longest Common Prefix
Path Length =  5
Shortest path: , 2399. Check Distances Between Same Letters, 1. Two Sum, Array, 989. Add to Array-Form of Integer, 2. Add Two Numbers
Path Length =  4
Shortest path: , 2399. Check Distances Between Same Letters, 1. Two Sum
Path Length =  1


## Single-Hop Connected Problems Coverage

In [34]:
# Get all problem nodes
plus_term = len(solved_questions)
total_problem_nodes = g.run("MATCH (n:Problem )RETURN n")
total_problem_nodes = total_problem_nodes.data()

In [35]:
total_problems = len(total_problem_nodes)
print(total_problems)

2478


In [36]:
# Get total 1-depth coverage across all solved problems to determine problem node % coverage
covered_questions = set()
for problem in solved_questions:
      source = str(problem['id']) + ". " + problem['title']
      func_string_c = f""" 
      MATCH (t:Problem {{title: '{source}'}})-[:SIMILAR_TO]-(n:Problem)
      RETURN n
      """
      covered_nodes = g.run(func_string_c)
      covered_nodes = covered_nodes.data()
      for node in covered_nodes:
        covered_questions.add(node['n']['title'])

covered_num = len(covered_questions)
print("Percent coverage at 1-depth neighbors for similar questions: ", covered_num/total_problems)

Percent coverage at 1-depth neighbors for similar questions:  0.014527845036319613


## N-hop problem coverage

In [ ]:
# Get total N-depth coverage across all solved problems to determine problem node % coverage
# This means that EVERY reachable node with no tag hops allowed
#STOPPED: TOOK TOO LONG TO RUN
covered_questions = set()
for problem in solved_questions:
      source = str(problem['id']) + ". " + problem['title']
      func_string_c1 = f""" 
      MATCH (t:Problem {{title: '{source}'}})-[:SIMILAR_TO*1..100]-()-[:SIMILAR_TO*1..100]->(n:Problem)
        RETURN distinct n
      """
      # print(func_string_c1)
      # func_string_c1 = f""" 
      # MATCH (t:Problem {{title: '{source}'}})-[:SIMILAR_TO|BELONGS_TO*1..100]-(n:Problem)
      #   RETURN distinct p
      # """
      covered_nodes = g.run(func_string_c1)
      covered_nodes = covered_nodes.data()
      for node in covered_nodes:
        covered_questions.add(node['n']['title'])

covered_num = len(covered_questions)
print("Percent coverage at N-depth neighbors for similar questions: ", covered_num/total_problems)